<a href="https://colab.research.google.com/github/wdavjr/wd-career-field-vs-occupation-eda/blob/main/wd_coffee_sales_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- ========= TITLE CELL ========= -->
<h1 align="center">WD • Kaggle Coffee Sales Analysis</h1>
<p align="center">
  Exploratory data analysis of the <a href="https://www.kaggle.com/datasets/navjotkaushal/coffee-sales-dataset">Coffee Sales Dataset</a><br>
  <sub>Python • Pandas • Matplotlib/Plotly • (Optional) Power BI</sub>
</p>

---

## About the Dataset
This dataset contains coffee shop transaction records, including details about sales, payment type, time of purchase, and customer preferences.  
It’s curated for data visualization, dashboarding, and business analytics in Power BI, Tableau, and Python (Matplotlib, Seaborn, Plotly).

**Columns**
- `hour_of_day` → 0–23 hour of purchase  
- `cash_type` → payment mode (cash/card)  
- `money` → transaction amount  
- `coffee_name` → e.g., Latte, Americano, Hot Chocolate  
- `Time_of_Day` → Morning / Afternoon / Night  
- `Weekday` → Mon–Sun  
- `Month_name` → Jan–Dec  
- `Weekdaysort` → 1–7 (Mon–Sun)  
- `Monthsort` → 1–12 (Jan–Dec)  
- `Date` → YYYY-MM-DD  
- `Time` → HH:MM:SS

**Good first visuals**
- Sales by coffee type
- Sales by hour of day / time of day
- Weekday & month seasonality
- Payment method split
- Revenue trends over time


In [146]:
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 15)

In [166]:
# Load and wrangle data
df = pd.read_csv('Coffe_sales.csv')

df = df.sort_values(['Date', 'Time_of_Day'], ascending=[False, True])

# Convert Date/Time columns to datetime objects
df['Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.normalize()

# trims microseconds via formatting
df['time_only'] = (
    pd.to_datetime(df['Time'].astype(str).str.strip(),
                   format='%H:%M:%S.%f', errors='coerce')
      .dt.strftime('%H:%M:%S')
)

# Realign column order
df = df[['Date', 'Time_of_Day', 'time_only', 'hour_of_day','coffee_name', 'money',
     'cash_type', 'Month_name', 'Monthsort', 'Weekday', 'Weekdaysort']].copy()

df['cash_type'] = df['cash_type'].str.title()

# Lower Snake-Case columns
df.columns = df.columns.str.lower()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3547 entries, 3543 to 10
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         3547 non-null   datetime64[ns]
 1   time_of_day  3547 non-null   object        
 2   time_only    3539 non-null   object        
 3   hour_of_day  3547 non-null   int64         
 4   coffee_name  3547 non-null   object        
 5   money        3547 non-null   float64       
 6   cash_type    3547 non-null   object        
 7   month_name   3547 non-null   object        
 8   monthsort    3547 non-null   int64         
 9   weekday      3547 non-null   object        
 10  weekdaysort  3547 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 332.5+ KB


In [167]:
# Display the first 5 rows
df.head()

,date,time_of_day,time_only,hour_of_day,coffee_name,money,cash_type,month_name,monthsort,weekday,weekdaysort
3543,2025-03-23,Afternoon,14:43:37,14,Cocoa,35.76,Card,Mar,3,Sun,7
3544,2025-03-23,Afternoon,14:44:16,14,Cocoa,35.76,Card,Mar,3,Sun,7
3545,2025-03-23,Afternoon,15:47:28,15,Americano,25.96,Card,Mar,3,Sun,7
3541,2025-03-23,Morning,10:07:11,10,Latte,35.76,Card,Mar,3,Sun,7
3542,2025-03-23,Morning,10:34:54,10,Cappuccino,35.76,Card,Mar,3,Sun,7


## Total Coffee Sales

In [168]:
df.describe()

,date,hour_of_day,money,monthsort,weekdaysort
count,3547,3547.000000,3547.000000,3547.000000,3547.000000
mean,2024-10-04 17:34:43.676346368,14.185791,31.645216,6.453905,3.845785
min,2024-03-01 00:00:00,6.000000,18.120000,1.000000,1.000000
25%,2024-07-17 12:00:00,10.000000,27.920000,3.000000,2.000000
50%,2024-10-10 00:00:00,14.000000,32.820000,7.000000,4.000000
75%,2025-01-11 00:00:00,18.000000,35.760000,10.000000,6.000000
max,2025-03-23 00:00:00,22.000000,38.700000,12.000000,7.000000
std,NaN,4.234010,4.877754,3.500754,1.971501


### What’s the date range, row count, and number of unique coffees?

In [181]:
# Get the date range
min_date = df['date'].dt.date.min()
max_date = df['date'].dt.date.max()

# Get the row count
row_count = len(df)

# Get the number of unique coffees
unique_coffee_count = df["coffee_name"].nunique()


narrative = f"""
The date range of the dataset is: {min_date} to {max_date}
The number of observations (row count) of the dataset is: {row_count: ,}
The number of number of unique of coffee types is: {unique_coffee_count}
"""

print(narrative)


The date range of the dataset is: 2024-03-01 to 2025-03-23
The number of observations (row count) of the dataset is:  3,547
The number of number of unique of coffee types is: 8



In [201]:
# Are there missing values in any column? Which ones and how many? hint: isna + sum per column
missing_column_values = df.isna().sum()
missing_column_values = missing_column_values[missing_column_values > 0]

print(missing_column_values)

time_only    8
dtype: int64


In [184]:
# Any invalid values: hour_of_day outside 0–23 or money <= 0? hint: boolean filters + counts





In [185]:
# Do Weekdaysort and Monthsort line up with Weekday/Month_name? hint: map dictionaries → compare → mismatch rate





In [186]:
# Does Time_of_Day agree with your own daypart from hour_of_day? hint: define bins → compare labels → % mismatch





In [ ]:
#



In [ ]:
#



In [ ]:
#

